In [1]:
import os
from imutils import paths
import random
import shutil
import numpy as np 

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K


2023-03-19 22:29:02.245145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 22:29:04.439184: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/haghob/Documents/ydays_project/envsp/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:
2023-03-19 22:29:04.439307: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/haghob/D

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam

In [4]:
import matplotlib
matplotlib.use("Agg")



from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

import tensorflow as tf



from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import argparse
import easydict

In [5]:
ORIG_INPUT_DATASET = "/home/haghob/Documents/ydays_project/breast_cancer_MRI/IDC_regular_ps50_idx5"

#Initialize the base path to the new directory that will contain our images after computing the training and testing split
BASE_PATH = "/home/haghob/Documents/ydays_project/breast_cancer_MRI/idc"

In [6]:
#Derive the training, validation, and testing directories
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

In [7]:
#Define the amount of data that will be used training
TRAIN_SPLIT = 0.8

#The amount of validation data will be a percentage of the *training* data
VAL_SPLIT = 0.1

In [8]:
#Grab the paths to all input images in the original input directory and shuffle them

imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

In [9]:
#Compute the training and testing split

i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

In [10]:
#We'll be using part of the training data for validation

i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

In [11]:
#Define the datasets that we'll be building

datasets = [("training", trainPaths, TRAIN_PATH),("validation", valPaths, VAL_PATH),
            ("testing", testPaths, TEST_PATH)] 

In [12]:
#Loop over the datasets

for (dType, imagePaths, baseOutput) in datasets:
    #Show which data split we are creating
    print("[INFO] building '{}' split".format(dType))
    
    #If the output base output directory does not exist, create it
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)
    
    #Loop over the input image paths
    for inputPath in imagePaths:
        
        #Extract the filename of the input image and extract the class label 0 for "negative" and 1 for "positive"
        filename = inputPath.split(os.path.sep)[-1]
        label = filename[-5:-4]
        
        #Build the path to the label directory
        labelPath = os.path.sep.join([baseOutput, label])
        
        # If the label output directory does not exist, create it
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)
        
        #Construct the path to the destination image and then copy the image itself
        p = os.path.sep.join([labelPath, filename])
        shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] building 'validation' split
[INFO] building 'testing' split


In [13]:
class CancerNet:
    @staticmethod
    def build(width, height, depth, classes):
        
        #Initialize the model along with the input shape to be"channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        #If we are using "channels first", update the input shape and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # CONV => RELU => POOL
        model.add(SeparableConv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        #(CONV => RELU => POOL) * 2
        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU => POOL) * 3
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(SeparableConv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        #First (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        #Softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [14]:
#tf.config.list_physical_devices()

os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [15]:
#Construct the argument parser and parse the arguments 

#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

args = easydict.EasyDict({"p": "plot.png"}) 

#Initialize our number of epochs, initial learning rate, and batch
# size

NUM_EPOCHS = 40
INIT_LR = 1e-3
BS = 32

#Determine the total number of image paths in training, validation, and testing directories
trainPaths = list(paths.list_images(TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

#Calculate the total number of training images in each class and initialize a dictionary to store the class weights
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = dict()

#Loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]
    #Initialize the training data augmentation object
trainAug = ImageDataGenerator(rescale=1 / 255.0, rotation_range=20, zoom_range=0.05,
    width_shift_range=0.1, height_shift_range=0.1, shear_range=0.05, horizontal_flip=True,
    vertical_flip=True, fill_mode="nearest")

# Initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

#Initialize the training generator
trainGen = trainAug.flow_from_directory(TRAIN_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb", shuffle=True, batch_size=BS)

#Initialize the validation generator
valGen = valAug.flow_from_directory(VAL_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb",shuffle=False, batch_size=BS)

#Initialize the testing generator
testGen = valAug.flow_from_directory(TEST_PATH, class_mode="categorical", target_size=(48, 48),
    color_mode="rgb", shuffle=False, batch_size=BS)

Found 199818 images belonging to 2 classes.
Found 22201 images belonging to 2 classes.
Found 55505 images belonging to 2 classes.


In [16]:
'''model = CancerNet.build(width=48, height=48, depth=3,classes=2)

#Here we are testing Adagrad as optimizer
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
#Fit the model

H = model.fit(x=trainGen, steps_per_epoch=totalTrain // BS,
    validation_data=valGen,validation_steps=totalVal // BS,
    class_weight=classWeight,epochs=NUM_EPOCHS)'''


from tensorflow.keras.optimizers import Adam

model = CancerNet.build(width=48, height=48, depth=3, classes=2)

opt = Adam(learning_rate=INIT_LR)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

H = model.fit(x=trainGen, steps_per_epoch=totalTrain // BS,
    validation_data=valGen, validation_steps=totalVal // BS,
    class_weight=classWeight, epochs=NUM_EPOCHS)



2023-03-19 22:30:56.093675: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2023-03-19 22:30:56.107244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: haghob
2023-03-19 22:30:56.107277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: haghob
2023-03-19 22:30:56.108651: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2023-03-19 22:30:56.109313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.13
2023-03-19 22:30:56.109347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 525.60.13 does not match DSO version 510.108.3 -- cannot find working devices in this configuration
2023-03-19 22:30:5

Epoch 1/40
6244/6244 [==============================] - 738s 118ms/step - loss: 0.5960 - accuracy: 0.8193 - val_loss: 0.4679 - val_accuracy: 0.8143
Epoch 2/40
6244/6244 [==============================] - 758s 121ms/step - loss: 0.5342 - accuracy: 0.8380 - val_loss: 0.4591 - val_accuracy: 0.8172
Epoch 3/40
6244/6244 [==============================] - 763s 122ms/step - loss: 0.5145 - accuracy: 0.8445 - val_loss: 0.5379 - val_accuracy: 0.7861
Epoch 4/40
6244/6244 [==============================] - 762s 122ms/step - loss: 0.5128 - accuracy: 0.8456 - val_loss: 0.6086 - val_accuracy: 0.7334
Epoch 5/40
6244/6244 [==============================] - 718s 115ms/step - loss: 0.5045 - accuracy: 0.8480 - val_loss: 0.3506 - val_accuracy: 0.8699
Epoch 6/40
6244/6244 [==============================] - 733s 117ms/step - loss: 0.4984 - accuracy: 0.8502 - val_loss: 0.7087 - val_accuracy: 0.7634
Epoch 7/40
6244/6244 [==============================] - 689s 110ms/step - loss: 0.4917 - accuracy: 0.8523 - val_

In [17]:
model.save('/home/haghob/Documents/ydays_project/CancerNet.h5') 
np.save('CancerNet.npy',H.history) 

In [18]:
model = keras.models.load_model('/home/haghob/Documents/ydays_project/CancerNet.h5')
history=np.load('CancerNet.npy',allow_pickle='TRUE').item()

#Reset the testing generator and then use our trained model to make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)

#For each image in the testing set we need to find the index of the label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

#Show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,target_names=testGen.class_indices.keys()))

In [25]:
# Reset the testing generator and then use our trained model to make predictions on the data
print("[INFO] evaluating network...")

testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)

#For each image in the testing set we need to find the index of the label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

#Show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,target_names=testGen.class_indices.keys()))

[INFO] evaluating network...
1735/1735 [==============================] - 60s 34ms/step
              precision    recall  f1-score   support

           0       0.96      0.83      0.89     39624
           1       0.68      0.91      0.78     15881

    accuracy                           0.85     55505
   macro avg       0.82      0.87      0.84     55505
weighted avg       0.88      0.85      0.86     55505



In [26]:
#Compute the confusion matrix and and use it to derive the raw accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

#Show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

[[32947  6677]
 [ 1417 14464]]
acc: 0.8542
sensitivity: 0.8315
specificity: 0.9108


In [30]:
#Plot the training loss and accuracy

N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()

<Figure size 640x480 with 0 Axes>

In [34]:
plt.plot(np.arange(0, N), history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history["val_loss"], label="val_loss")
#plt.plot(np.arange(0, N), history["acc"], label="train_acc")
#plt.plot(np.arange(0, N), history["val_acc"], label="val_acc")  

In [35]:
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["p"])

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separable  (None, 48, 48, 32)       155       
 Conv2D)                                                         
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                        